In [21]:
import heapq
from collections import defaultdict

# 削除可能ヒープ
# self.heap: ヒープ
# self.delete_heap: 削除予約待ちのヒープ
# self.item_cnt: defaultdict(key:heapの要素からdelete_heapを除いた要素，value:その要素が何個あるか)
# len(): ヒープから削除予約待ちのヒープを除いた要素数
# __repr__(): ヒープから削除予約待ちのヒープを除いた要素をリストにして返す
# __iter__(): ヒープから削除予約待ちのヒープを除いた要素をイテレータとして返す
# pop(): ヒープの最小値を取り出す
# push(item): ヒープにitemを追加する
# remove(item): ヒープからitemを削除する(内部的には削除予約しておいて，その要素がヒープの先頭にきたら削除する)
# get_item_cnt(item): ヒープ内のitemの個数を返す
class DeletableHeap:
    def __init__(self,heap):
        self.heap = heap
        self.delete_heap = []
        self.item_cnt = defaultdict(int)
        for item in heap:
            self.item_cnt[item] += 1
    def __len__(self):
        return len(self.heap) - len(self.delete_heap)
    def __repr__(self):
        return str(list(self))
    def __iter__(self):
        j = 0
        for item in self.heap:
            if j < len(self.delete_heap):
                if item == self.delete_heap[j]:
                    j += 1
                    continue
            yield item
    def pop(self):
        while self.heap and self.delete_heap:
            if self.heap[0] == self.delete_heap[0]:
                heapq.heappop(self.heap)
                heapq.heappop(self.delete_heap)
            else:
                break
        return_item = heapq.heappop(self.heap)
        self.item_cnt[return_item] -= 1
        return return_item
    def push(self,item):
        heapq.heappush(self.heap,item)
        self.item_cnt[item] += 1
    def remove(self,item):
        heapq.heappush(self.delete_heap,item)
        self.item_cnt[item] -= 1
        while self.heap and self.delete_heap:
            if self.heap[0] == self.delete_heap[0]:
                heapq.heappop(self.heap)
                heapq.heappop(self.delete_heap)
            else:
                break
    def get_item_cnt(self,item):
        return self.item_cnt[item]

def main():
    N,K,Q = map(int,input().split())
    # 大きい方からK個の数が入ってるヒープ，最小値を取り出せるようにしとく
    big_heap = DeletableHeap([0]*K)
    # 小さい方からN-K個の数が入ってるヒープ，最大値を取り出せるようにしとく(追加と取り出しに-1をかける)
    small_heap = DeletableHeap([0]*(N-K))
    A = [0]*N
    ans = 0
    for _ in [0]*Q:
        x,y = map(int,input().split())
        x -= 1
        # Aのx番目の要素が大きい方のヒープにある場合
        if big_heap.get_item_cnt(A[x]):
            # 変更前の値をansから引いて，変更後の値をansに足す
            ans = ans - A[x] + y
            # big_heapからA[x]を削除
            big_heap.remove(A[x])
            # big_heapにyを追加
            big_heap.push(y)
        # Aのx番目の要素が小さい方のヒープにある場合
        else:
            # small_heapからA[x]を削除
            small_heap.remove(-A[x])
            # small_heapにyを追加
            small_heap.push(-y)
        # big_heapの最小値がsmall_heapの最大値より小さい場合
        if small_heap:
            if big_heap.heap[0] < -small_heap.heap[0]:
                # big_heapの最小値をansから引いてsmall_heapに移動させる
                min_big = big_heap.pop()
                small_heap.push(-min_big)
                ans -= min_big
                # small_heapの最大値をansに足してbig_heapに移動させる
                max_small = -small_heap.pop()
                big_heap.push(max_small)
                ans += max_small
        # Aのx番目の要素をyに更新
        A[x] = y
        print(ans)
    
if __name__ == "__main__":
    main()

5
6
8
8
15
13
13
11
1
0
